In [4]:
import sqlite3
import pandas as pd
import numpy as np
con = sqlite3.connect("dump.db")
cur = con.cursor()
UserPath = pd.read_sql("select * from UserPath", con)[:10000]

In [5]:
#это дропнуть для нормальной бд
new_actions = ['Like', 'Dislike', 'Favourite']
all_actions = ['EventShow', 'SuccessPurchase'] + new_actions
np.random.seed(42)  
random_actions = np.random.choice(all_actions, size=len(UserPath))
UserPath['Act'] = random_actions
#это дропнуть для нормальной бд

In [6]:
interaction_matrix = pd.DataFrame(0, index=UserPath['UserID'].unique(), columns=UserPath['EventID'].unique())
for _, row in UserPath.iterrows():
    if row['Act'] == 'Dislike':
        interaction_matrix.loc[row['UserID'], row['EventID']] = -1
    elif row['Act'] == 'EventPreBuy':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 1
    elif row['Act'] == 'Favourite':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 2
    elif row['Act'] == 'EventPreBuy':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 3
    elif row['Act'] == 'BuyWithContact':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 4
    elif row['Act'] == 'Like':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 5
    elif row['Act'] == 'SuccessPurchase':
        interaction_matrix.loc[row['UserID'], row['EventID']] = 6

interaction_matrix = interaction_matrix.stack().reset_index()
interaction_matrix.columns = ['user_id', 'event_id', 'action']
interaction_matrix

,user_id,event_id,action
0,6eaafef9d6c89ac6f9d894273710e623,93617e256881c731fef1e44d75efd4a7,-1
1,6eaafef9d6c89ac6f9d894273710e623,263d7f23720cb4c3e49e8f72d4da05a4,0
2,6eaafef9d6c89ac6f9d894273710e623,1727bc129a45f77c44734f22a4d9ac2a,0
3,6eaafef9d6c89ac6f9d894273710e623,406d4c53608352ab96fe9757fcaeede5,0
4,6eaafef9d6c89ac6f9d894273710e623,0f0e98b8568dd945a9b6cf83e36faf0c,0
...,...,...,...
31440745,df2d613647d4d0e60ef08375e9f5dc74,c93862bb48de68a74d6c54ca95fad122,0
31440746,df2d613647d4d0e60ef08375e9f5dc74,e9936d659692a672e05c8a087cf1798a,0
31440747,df2d613647d4d0e60ef08375e9f5dc74,63964f60a2b6d46dcf4eb262b4a00053,0
31440748,df2d613647d4d0e60ef08375e9f5dc74,780c239040ca41405505813658d6a6ac,0


In [7]:
interaction_matrix = interaction_matrix.pivot_table(index='user_id', columns='event_id', values='action', fill_value=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc_matrix = sc.fit_transform(interaction_matrix)

In [9]:
df = pd.DataFrame(sc_matrix, index=interaction_matrix.index, columns=interaction_matrix.columns)

In [10]:
def knn(df, user_id, n_recommendations=5, n_similar = 100):
    s = {}
    
    for i in df.index:
        if i != user_id: 
            s[(df.loc[user_id] - df.loc[i]).sum()] = i

    similar_users = [user for _, user in sorted(s.items())[:n_similar]]

    recommendations = {}
    for similar_user in similar_users:
        for event_id in df.columns:
            if df.loc[user_id][event_id] == 0:  
                if event_id not in recommendations:
                        recommendations[event_id] = df.loc[similar_user, event_id]
                else:
                    recommendations[event_id] += df.loc[similar_user, event_id]
    recommended_events = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n_recommendations]
    
    return recommended_events


In [11]:
knn(interaction_matrix, '000a1dfe6f4f3018eb805e8115a0b5fc')

[('5a730579ebe031843c2cda200a47bb67', np.float64(10.0)),
 ('4bc94de7bab27a1c3a9cd3ba68199f55', np.float64(8.0)),
 ('05adf2abceb1050b43f7b96a0f15f4a6', np.float64(6.0)),
 ('1afba3930b0ec2d4e100f056e20d9d65', np.float64(6.0)),
 ('1ca1af55d2263e9139ba774ffff9dc0c', np.float64(6.0))]